In [ ]:
import pandas as pd
import datetime
import numpy as np
import altair as alt
import fidap
import config
import geopandas as gpd
from shapely import wkt
import matplotlib.pyplot as plt

# instantiate connection to fidap
fidap = fidap.fidap_client(api_key = config.api_key)

In [ ]:
btc_corr = fidap.sql("""
WITH BTC AS (
SELECT CAST(Open_Timr AS STRING) AS b_date, symbol, close,
LAG(close, 1) OVER (PARTITION BY symbol ORDER BY date ASC) AS previous_day_close,
ROUND((close/LAG(close, 1) OVER (PARTITION BY symbol ORDER BY date ASC)) - 1,2) AS pct_over_previous_day_close
FROM fidap-301014.binance.binance
WHERE symbol IN ('BTC/USDT')
ORDER BY date, symbol), 

ETH AS (
SELECT CAST(date AS STRING) AS e_date, symbol, close,
LAG(close, 1) OVER (PARTITION BY symbol ORDER BY date ASC) AS previous_day_close,
ROUND((close/LAG(close, 1) OVER (PARTITION BY symbol ORDER BY date ASC)) - 1,2) AS pct_over_previous_day_close
FROM fidap-301014.binance.klines_1day
WHERE symbol IN ('ETH/USDT')
ORDER BY date, symbol
)

SELECT CORR(b.pct_over_previous_day_close, e.pct_over_previous_day_close) AS corr_eth_btc
FROM ETH AS e 
INNER JOIN BTC AS b 
ON b_date = e_date 
WHERE e.pct_over_previous_day_close IS NOT NULL
""")

In [ ]:
highly_traded_pairs = fidap.sql("""
SELECT PAIR_NAME, VOLUME, CAST(TIMESTAMP_MILLIS(CAST(OPEN_TIME AS INT64)) AS STRING) AS Open_Date
FROM fidap-301014.binance.klines_1month
ORDER BY Open_Date DESC, VOLUME DESC
LIMIT 10
""")

In [ ]:
most_pops_pairs = fidap.sql("""
WITH ten_most_traded_pair AS (
SELECT PAIR_NAME, Quote_asset_volume, CAST(TIMESTAMP_MILLIS(Open_Time) AS STRING) AS Open_Date
FROM fidap-301014.binance.klines_1day
ORDER BY Open_Date DESC, Quote_asset_volume DESC
LIMIT 10) 

SELECT b.PAIR_NAME, Number_of_trades, SUBSTR(CAST(TIMESTAMP_MILLIS(a.Open_Time) AS STRING),1,10) AS Open_Date
FROM fidap-301014.binance.klines_1day AS a
INNER JOIN ten_most_traded_pair AS b
ON a.PAIR_NAME = b.PAIR_NAME
WHERE a.Open_Time > UNIX_MILLIS(TIMESTAMP "2021-09-21 00:00:00+00")
ORDER BY b.PAIR_NAME, Open_Date
""")